In [1]:
import torch, torchvision
torchvision.disable_beta_transforms_warning()
import time
import numpy as np

In [2]:
def save(self, destination):
    from torch import package

    #internal modules
    MODULES = ['datasets', 'traininglib']
    
    if isinstance(destination, str):
        destination = time.strftime(destination)
        if not destination.endswith('.pt.zip'):
            destination += '.pt.zip'

    with package.PackageExporter(destination) as exp:
        interns = [__name__.split('.')[-1]]+MODULES
        exp.intern(interns)
        exp.extern('**', exclude=['torchvision.**'])
        externs = ['torchvision.ops.**', 'torchvision.datasets.**', 'torchvision.io.**']
        exp.intern('torchvision.**', exclude=externs)
        exp.extern(externs)
        exp.intern('torchvision.models.detection.**')
        # force inclusion of internal modules + re-save if importlib.reload'ed
        for m in MODULES:
            exp.save_module(m, dependencies=True)
        exp.save_module('modellib', dependencies=True)
        exp.save_pickle('model', 'model.pkl', self)
        exp.save_text('model', 'class_list.txt', '\n'.join(self.class_list))
    return destination

In [3]:
from modellib import DuckDetector
model = DuckDetector(classes_of_interest=[ # class order must match the label_dict from training
                                    'AMCO', 'GADW', 'GWTE', 'MALL', 'NOPI', 'NSHO', 'REDH', 'RNDU'])

basemodel_pt_zip = save(self=model, destination="basemodel.pt.zip")

In [4]:
imp = torch.package.PackageImporter(basemodel_pt_zip)
print(imp.file_structure())

─── basemodel.pt.zip
    ├── .data
    │   ├── 0.storage
    │   ├── 1.storage
    │   ├── 10.storage
    │   ├── 11.storage
    │   ├── 12.storage
    │   ├── 13.storage
    │   ├── 14.storage
    │   ├── 15.storage
    │   ├── 16.storage
    │   ├── 17.storage
    │   ├── 18.storage
    │   ├── 19.storage
    │   ├── 2.storage
    │   ├── 20.storage
    │   ├── 21.storage
    │   ├── 22.storage
    │   ├── 23.storage
    │   ├── 24.storage
    │   ├── 25.storage
    │   ├── 26.storage
    │   ├── 27.storage
    │   ├── 28.storage
    │   ├── 29.storage
    │   ├── 3.storage
    │   ├── 30.storage
    │   ├── 31.storage
    │   ├── 32.storage
    │   ├── 33.storage
    │   ├── 34.storage
    │   ├── 35.storage
    │   ├── 36.storage
    │   ├── 37.storage
    │   ├── 38.storage
    │   ├── 39.storage
    │   ├── 4.storage
    │   ├── 40.storage
    │   ├── 41.storage
    │   ├── 42.storage
    │   ├── 43.storage
    │   ├── 44.storage
    │   ├── 45.storage
    │   ├── 46.storage
    

### <center>Test that packaged model opens</center>

In [5]:
def load_model(file_path:str, model_name:str) -> "torch.nn.Module":
            return torch.package.PackageImporter(file_path).load_pickle(model_name, f'{model_name}.pkl', map_location='cpu')

In [7]:
model = load_model(file_path='basemodel.pt.zip', model_name='model')
model

DuckDetector(
  (detector): Detector(
    (basemodel): SSD(
      (backbone): SSDFeatureExtractorVGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
          (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (6): ReLU(inplace=True)
          (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): ReLU(inplace=True)
          (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): ReLU(inplace=True)
          (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (13): ReLU(

### <center> Process sample images </center>

In [8]:
model.class_list

['AMCO', 'GADW', 'GWTE', 'MALL', 'NOPI', 'NSHO', 'REDH', 'RNDU']

In [9]:
prediction = model.process_image('C:/Users/zack/Desktop/DuckNet_Data/Images_Test/DJI_20230105170403_0091_Z.JPG')

output {'boxes': tensor([[ 68.5479, 184.3041,  90.7941, 207.2784],
        [ 70.2120, 281.0377,  93.8413, 298.3578],
        [103.5943,  68.6801, 122.8241,  94.4029],
        [138.0472, 137.7994, 156.6600, 165.6302],
        [202.8893,  57.3380, 219.7253,  85.4486],
        [  5.7140, 167.2711,  29.8765, 190.3628],
        [  5.7140, 167.2711,  29.8765, 190.3628],
        [ 67.5346, 183.5834,  90.3303, 208.0754],
        [137.7279, 138.0132, 156.7391, 165.6858],
        [103.1496,  69.0710, 123.3665,  94.2176],
        [202.9565,  57.7597, 219.5202,  85.8412],
        [ 70.2120, 281.0377,  93.8413, 298.3578],
        [103.5943,  68.6801, 122.8241,  94.4029],
        [138.0472, 137.7994, 156.6600, 165.6302],
        [ 70.0272, 281.0748,  94.3444, 298.6465],
        [  4.2455, 166.5855,  30.7530, 190.2510],
        [202.9565,  57.7597, 219.5202,  85.8412],
        [137.7279, 138.0132, 156.7391, 165.6858],
        [202.9565,  57.7597, 219.5202,  85.8412],
        [  4.2455, 166.5855,  30.

### <center>Train model</center> 

In [10]:
# # import datasets module
# from datasets import *
file_path = 'basemodel.pt.zip'
module = 'datasets'

datasets = torch.package.PackageImporter(file_path).import_module(module)

In [ ]:
sample_dataset = datasets.DetectionDataset(jpgfiles = 'C:/Users/zack/Desktop/DuckNet_Data/Images_Test/',
                                           jsonfiles = 'C:/Users/zack/Desktop/DuckNet_Data/LabelMe_Annotations_Test/',
                                           augment  = True,
                                           negative_classes = [])    

In [ ]:
image, target = sample_dataset[0]

In [ ]:
print(target)

In [ ]:
import matplotlib.pyplot as plt

# turn model.class_list into a dictionary
label_dict = {i+1: model.class_list[i] for i in range(len(model.class_list))}

# distinct colors 
distinct_colors = ['#f032e6', '#ffffff', '#ffe119', '#3cb44b', '#42d4f4',
                    '#f58231', '#e6194B', '#dcbeff', '#469990', '#4363d8']

# label color map for plotting color-coded boxes by class
label_color_map = {k: distinct_colors[i] for i, k in enumerate(label_dict.keys())}

# classes are values in label_dict
classes = list(label_dict.values())

# reverse label dictionary for mapping predictions to classes
rev_label_dict = {v: k for k, v in label_dict.items()}

# distinct colors 
distinct_colors = ['#f032e6', '#ffffff', '#ffe119', '#3cb44b', '#42d4f4',
                    '#f58231', '#e6194B', '#dcbeff', '#469990', '#4363d8']

# label color map for plotting color-coded boxes by class
label_color_map = {k: distinct_colors[i] for i, k in enumerate(label_dict.keys())}

# function for reshaping boxes 
def get_box(boxes):
    boxes = np.array(boxes)
    boxes = boxes.astype('float').reshape(-1, 4)
    if boxes.shape[0] == 1 : return boxes
    return np.squeeze(boxes)


# function for plotting image
def img_show(image, ax = None, figsize = (6, 9)):
    if ax is None:
        fig, ax = plt.subplots(figsize = figsize)
    ax.xaxis.tick_top()
    ax.imshow(image)
    return ax
 

def plot_bbox(ax, boxes, labels):
    # add box to the image and use label_color_map to color-code by bounding box class if exists else 'black'
    ax.add_patch(plt.Rectangle((boxes[:, 0], boxes[:, 1]), boxes[:, 2] - boxes[:, 0], boxes[:, 3] - boxes[:, 1],
                    fill = False,
                    color = label_color_map[labels.item()] if labels.item() in label_color_map else 'black', 
                    linewidth = 1.5))
    # add label text to bounding box using label_dict if label exists else labels
    ax.text(boxes[:, 2], boxes[:, 3], 
            (label_dict[labels.item()] if labels.item() in label_dict else None),
            fontsize = 8,
            bbox = dict(facecolor = 'white', alpha = 0.8, pad = 0, edgecolor = 'none'),
            color = 'black')


# function for plotting all boxes and labels on the image using get_polygon, img_show, and plot_mask functions
def plot_detections(image, boxes, labels, ax = None):
    ax = img_show(image.permute(1, 2, 0), ax = ax)
    for i in range(len(boxes)):
        box = get_box(boxes[i])
        plot_bbox(ax, box, labels[i])

In [ ]:
plot_detections(image, target['boxes'], target['labels']) # Sample anno data

In [ ]:
sample_dataloader = datasets.create_dataloader(sample_dataset, batch_size = 16, shuffle = True)

In [ ]:
images, targets = next(iter(sample_dataloader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

In [ ]:
# Plot the all samples from batch in a grid of subplots. 
plt.figure(figsize = (8, 32))
for i in range(8):
    ax = plt.subplot(8, 2, 1 + i)
    plot_detections(images[i], targets[i]['boxes'], targets[i]['labels'], ax = ax)
    plt.axis('off')
    plt.title(f"Sample {i + 1}")

plt.show()

In [ ]:
model.class_list

In [ ]:
model = load_model(file_path='basemodel.pt.zip', model_name='model')
model.start_training_detector(imagefiles_train = 'C:/Users/zack/Desktop/DuckNet_Data/Images_Train/',
                              jsonfiles_train = 'C:/Users/zack/Desktop/DuckNet_Data/LabelMe_Annotations_Train/',
                              imagefiles_test = 'C:/Users/zack/Desktop/DuckNet_Data/Images_Test/',
                              jsonfiles_test = 'C:/Users/zack/Desktop/DuckNet_Data/LabelMe_Annotations_Test/',
                              negative_classes = [],
                              epochs = 10,
                              lr = 0.0001,
                              )

### <center> Converting Darwin JSON to LabelMe JSON </center>

In [ ]:
# import json 

# def get_imagename_from_jsonfile(jsonfile):
#     with open(jsonfile, 'r') as j:
#         jsondata = json.loads(j.read())
#     return jsondata['item']['slots'][0]['source_files'][0]['file_name']


# def get_boxes_from_jsonfile(jsonfile):
#     '''Reads bounding boxes from a DARWIN json file and returns them as a (Nx4) array'''
#     with open(jsonfile, 'r') as j:
#         jsondata = json.loads(j.read())
        
#     boxes = []
#     for i in range(len(jsondata['annotations'])):
#         box = [[jsondata['annotations'][i]['bounding_box']['x'], # xmin
#                 jsondata['annotations'][i]['bounding_box']['y']], # ymin
#                 [jsondata['annotations'][i]['bounding_box']['x']+jsondata['annotations'][i]['bounding_box']['w'], # xmax
#                 jsondata['annotations'][i]['bounding_box']['y']+jsondata['annotations'][i]['bounding_box']['h']]] # ymax
#         boxes.append(box)
#     return boxes # return as (Nx4) array of bounding


# def get_labels_from_jsonfile(jsonfile):
#     '''Reads a list of labels in a DARWIN json file.'''
#     with open(jsonfile, 'r') as j:
#         jsondata = json.loads(j.read())
#     return [ a['name'] for a in jsondata['annotations'] ]
 

# def get_imagesize_from_jsonfile(jsonfile):
#     with open(jsonfile, 'r') as j:
#         jsondata = json.loads(j.read())
#     return (jsondata['item']['slots'][0]['height'], jsondata['item']['slots'][0]['width'])


# def darwin_to_labelme_json(jsondata):
#     # convert darwin json to labelme json format. 

#     # labelme json should have following format:
#     # {'version': '4.5.6',
#     #  'flags': {},
#     #  'shapes': [
#     #      {
#     #       'label': 'duck',
#     #       'points': [[xmin, ymin], [xmax, ymax]],
#     #       'group_id': null,
#     #       'shape_type': 'rectangle',
#     #       'flags': {}
#     #      },
#     #      ...
#     #  ],
#     # 'imagePath': 'path/to/image/file',
#     # 'imageData': 'base64 encoded image data',
#     # 'imageHeight': 480,
#     # 'imageWidth': 640}

#     # ignore the 'imageData' field. 

#     image_name = get_imagename_from_jsonfile(jsondata)
#     boxes = get_boxes_from_jsonfile(jsondata)
#     labels = get_labels_from_jsonfile(jsondata)
#     image_size = get_imagesize_from_jsonfile(jsondata)

#     shapes = []
#     for i in range(len(labels)):
#         shape = {'label': labels[i],
#                     'points': boxes[i],
#                     'group_id': 'null',
#                     'shape_type': 'rectangle',
#                     'flags': {}}
#         shapes.append(shape)

#     for i in range(len(labels)):
#         labelme_json = {'version': '4.5.6',
#                         'flags': {},
#                         'shapes': shapes,
#                         'imagePath': image_name,
#                         'imageData': '',
#                         'imageHeight': image_size[0],
#                         'imageWidth': image_size[1]}
#     return labelme_json

# jsonfile = 'C:/Users/zack/Desktop/DuckNet_Data/Annotations_Test/DJI_20211215103949_0003_Z.json'
# labelme_json = darwin_to_labelme_json(jsonfile)
# labelme_json

In [ ]:
# import os

# dir = 'C:/Users/zack/Desktop/DuckNet_Data/'

# # create two new folders in dir: LabelMe_Annotations_Test and LabelMe_Annotations_Train
# os.makedirs(dir + 'LabelMe_Annotations_Test', exist_ok = True)
# os.makedirs(dir + 'LabelMe_Annotations_Train', exist_ok = True)

# # convert all json files in Annotations_Test to labelme json format and save them in LabelMe_Annotations_Test
# for jsonfile in os.listdir(dir + 'Annotations_Test/'):
#     labelme_json = darwin_to_labelme_json(dir + 'Annotations_Test/' + jsonfile)
#     with open(dir + 'LabelMe_Annotations_Test/' + jsonfile, 'w') as j:
#         json.dump(labelme_json, j)

# # convert all json files in Annotations_Train to labelme json format and save them in LabelMe_Annotations_Train
# for jsonfile in os.listdir(dir + 'Annotations_Train/'):
#     labelme_json = darwin_to_labelme_json(dir + 'Annotations_Train/' + jsonfile)
#     with open(dir + 'LabelMe_Annotations_Train/' + jsonfile, 'w') as j:
#         json.dump(labelme_json, j)